# Block VQE by Qubit

Here we will split the Hilbert space into two groups, one which will be calculated classically and the other with will be calculated using block-VQE.  Instead of splitting the Hilbert space into spin-up and spin-down as we did before, here I will split the Hilbert space into general sets of qubits.  

The wavefunction is divided into classical and quantum parts
$$
    |\psi> = \sum_c \alpha_c |c>|\psi_{qc}>
$$
where $|c>$ is a basis state in the classical section and $|\psi_q>$ is the wavefunction in the quantum section.  The weights $\alpha_c$ have to be solved classically while the wavefunction $|\psi_q>$ will be updated using VQE.  

For the Hamiltonian we will take the Hubbard model.  Using the Jordan-Wigner transformation, we have,
$$ 
    H = t \sum_{i\sigma} \left(X_{i\sigma}X_{i+1\sigma} + Y_{i\sigma} Y_{i+1\sigma}\right) + U \sum_i Z_{i\uparrow} Z_{i\downarrow}
$$

Let us take the first $N_c$ qubits to be in the classical sector.  Then we can take energy expectation values as,
\begin{equation}
\begin{split}
&<\psi|H|\psi> = \sum_{cc'}\alpha_c\alpha_c'<\psi_{qc'}|<c'| H |c> |\psi_{qc}>=
\\
& \sum_{cc'\sigma}\alpha_c\alpha_c't \left( \sum_{i=0}^{N_c-2}  <c'|\left(X_{i\sigma}X_{i+1\sigma} + Y_{i\sigma} Y_{i+1\sigma}\right)|c><\psi_{qc'}|\psi_{qc}>  + \delta_{cc'}\sum_{i=N_c+1}^{N} <\psi_{qc'}| \ \left(X_{i\sigma}X_{i+1\sigma} + Y_{i\sigma} Y_{i+1\sigma}\right) |\psi_{qc}> \right) 
\\
&+ \sum_{cc'\sigma}\alpha_c\alpha_c' t \left( <c'|X_{N_c-1\sigma}|c><\psi_{qc'}|X_{N_c\sigma}|\psi_{qc}> + <c'|Y_{N_c-1\sigma}|c> <\psi_{qc'}|Y_{N_c\sigma}|\psi_{qc}>\right) 
\\
&+ \sum_{cc'\sigma}\alpha_c\alpha_c' t \left( <c'|X_{0\sigma}|c><\psi_{qc'}|X_{N\sigma}|\psi_{qc}> + <c'|Y_{0\sigma}|c> <\psi_{qc'}|Y_{N\sigma}|\psi_{qc}>\right) 
\\
&+\sum_{cc'}\alpha_c\alpha_c'U \left(\sum_{i=0}^{N_c-1} <c'|Z_{i\uparrow} Z_{i\downarrow}|c><\psi_{qc'}|\psi_{qc}> + \delta_{cc'}\sum_{i=0}^{N_c-1} <\psi_{qc'}|Z_{i\uparrow} Z_{i\downarrow}|\psi_{qc}> \right)
\end{split}
\end{equation}

Anything in the classical brakets are calculated classically while anything in the quantum brackets are calculated using quantum hardware.  Notice that some of the quantum brakets are wieghted by classical brackets.  This is not a problem.  

## Classical part

### Define Pauli Matrices

Let us define the Pauli matrices so we can do the classical calculations.

In [40]:
import qiskit.quantum_info as qi

def X(i,s,N):
    label = ['I' for i in range(2*N)]
    label[i+s*N] = 'X'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Y(i,s,N):
    label = ['I' for i in range(2*N)]
    label[i+s*N] = 'Y'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

def Z(i,s,N):
    label = ['I' for i in range(2*N)]
    label[i+s*N] = 'Z'
    label = ''.join(label)
    return qi.Operator.from_label(label).data

### Generate the states

We need a function which gives you the classical bassi for a given number of qubits.

In [41]:
# A function to print the state given the numerical represenations
def bi(num,N):
    bi = bin(num)
    out = []
    Sdiff = 2*N - len(bi) + 2
    for i in range(0,Sdiff):
        out.append(0)
    for i in range(2,len(bi)):
        out.append(int(bi[i]))
    return out

# A function which retruns the numerical representation of states given N
def states(N):
    out = [i for i in range(0,4**N)]
    return out

# A function to print the basis vectors given the number of qubits N
def vecs(N):
    out = []
    for i in range(4**N):
        v = [0 for i in range(4**N)]
        v[i] = 1
        out.append(v)
    return out

In [58]:
import numpy as np

def Mdot(Ol):
    out = Ol[0]
    for i in range(1,len(Ol)):
        out = np.dot(Ol[i],out)
    return out

def bkt(y1,O,y2):
    return Mdot([np.conjugate(y1),O,y2])


### Calculating the classical brakets

We can calculate all of the classical brakets before we start VQE.  Once we have the value of each braket we simply store it as a weight for the various quantum brakets.  There are five types of classical brakets we need to calculate.

\begin{equation}
\begin{split}
& T_{c,c'} =  t<c'|\sum_{\sigma}\sum_{i=0}^{N_c-2}\left(X_{i\sigma}X_{i+1\sigma} + Y_{i\sigma} Y_{i+1\sigma}\right)|c> 
\\
& B1^x_{c,c'} =  t<c'|\sum_{\sigma}X_{N_c-1\sigma}|c>
\\
&B1^y_{c,c'} = t<c'|\sum_{\sigma}Y_{N_c-1\sigma}|c> 
\\
& B0^x_{c,c'} =  t<c'|\sum_{\sigma}X_{0\sigma}|c>
\\
&B0^y_{c,c'} = t<c'|\sum_{\sigma}Y_{0\sigma}|c> 
\\
&U_{c,c'} = U<c'|\sum_{\sigma}\sum_{i=0}^{N_c-1} Z_{i\uparrow} Z_{i\downarrow}|c>
\end{split}
\end{equation}

In [117]:
def T(t,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    for i in range(0,Nc-1):
        for s in range(0,2):
            O = Mdot([X(i,s,Nc),X(i+1,s,Nc)]) + Mdot([Y(i,s,Nc),Y(i+1,s,Nc)])
            out += bkt(v[c],t*O,v[cc])
    return out

def B1x(t,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    for s in range(0,2):
        O = X(Nc-1,s,Nc)
        out += bkt(v[c],t*O,v[cc])
    return out

def B1y(t,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    for s in range(0,2):
        O = Y(Nc-1,s,Nc)
        out += bkt(v[c],t*O,v[cc])
    return out

def B0x(t,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    for s in range(0,2):
        O = X(0,s,Nc)
        out += bkt(v[c],t*O,v[cc])
    return out

def B0y(t,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    for s in range(0,2):
        O = X(0,s,Nc)
        out += bkt(v[c],t*O,v[cc])
    return out

def U(u,c,cc,Nc):
    out = 0
    v = vecs(Nc)
    for i in range(0,Nc):
        O = Mdot([Z(i,0,Nc),Z(i,1,Nc)])
        out += bkt(v[c],u*O,v[cc])
    return out

We Can use these to rewrite the Hamiltoniain.  I will shift the indicies so $ N_c \rightarrow 0 $ and $N \rightarrow N-N_c = N_q $

\begin{equation}
\begin{split}
H_{c,c'}=& T_{c,c'}+ U_{c,c'} + t\delta_{cc'}\sum_{i=0}^{N_q} \sum_{\sigma} \left(X_{i\sigma}X_{i+1\sigma} + Y_{i\sigma} Y_{i+1\sigma}\right) + U\delta_{cc'}\sum_{i=0}^{N_q} Z_{i\uparrow} Z_{i\downarrow}
\\
&+\sum_{\sigma}  \left(B^{1x}_{c,c'}X_{0\sigma}+ B^{1y}_{c,c'} Y_{0\sigma}+  B^{0x}_{c,c'}X_{N_q\sigma} + B^{0y}_{c,c'} Y_{N_q\sigma} \right)
\end{split}
\end{equation}